# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [7]:
import pandas as pd

url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df_marketing_customer = pd.read_csv(url)
df_marketing_customer.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [9]:
df_marketing_customer.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

In [28]:
df_marketing_customer_filtered = df_marketing_customer
grouped_response = df_marketing_customer.groupby('Response')
grouped_response.size()


Response
No     8813
Yes    1466
dtype: int64

In [30]:
grouped_claim = df_marketing_customer.groupby('Total Claim Amount')


In [40]:
def claim_group(total_claim_amount):
    if total_claim_amount >= 1000:
        return "High"
    else:
        return "Low"

df_marketing_customer["claim_group"] = df_marketing_customer["Total Claim Amount"].apply(claim_group)
df_marketing_customer["claim_group"].value_counts()

claim_group
Low     10419
High      491
Name: count, dtype: int64

In [44]:
grouped_all = df_marketing_customer.groupby(['Response', 'Total Claim Amount'])['Total Claim Amount'].sum()

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [54]:
df_marketing_customer

df_claim_amount = grouped_response.get_group("Yes")

df_claim_amount.groupby(["Gender", "Policy Type"])["Total Claim Amount"].mean()

# 
#
#

Gender  Policy Type   
F       Corporate Auto    433.738499
        Personal Auto     452.965929
        Special Auto      453.280164
M       Corporate Auto    408.582459
        Personal Auto     457.010178
        Special Auto      429.527942
Name: Total Claim Amount, dtype: float64

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [64]:
state_grouped = df_marketing_customer.groupby("State")
state_grouped.value_counts()

State       Unnamed: 0  Customer  Customer Lifetime Value  Response  Coverage  Education             Effective To Date  EmploymentStatus  Gender  Income  Location Code  Marital Status  Monthly Premium Auto  Months Since Last Claim  Months Since Policy Inception  Number of Open Complaints  Number of Policies  Policy Type     Policy        Renew Offer Type  Sales Channel  Total Claim Amount  Vehicle Class  Vehicle Size  Vehicle Type  claim_group
Arizona     10          HG93801   5154.764074              No        Extended  High School or Below  1/2/11             Employed          M       82730   Urban          Married         127                   13.0                     31                             0.0                        1                   Corporate Auto  Corporate L3  Offer2            Branch         442.521087          SUV            Large         A             Low            1
            26          XF15789   8869.619888              No        Basic     College             

In [56]:
df_marketing_customer.groupby(["State", "Customer"])["Number of Policies"].count()

State       Customer
Arizona     AA35519     1
            AA56476     2
            AA71604     2
            AB45325     1
            AB98823     1
                       ..
Washington  ZX85480     1
            ZY20779     1
            ZY90118     1
            ZZ22047     1
            ZZ41158     1
Name: Number of Policies, Length: 9134, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [66]:
df_marketing_customer.groupby(["Gender", "Education"])["Customer Lifetime Value"].median()


Gender  Education           
F       Bachelor                5640.505303
        College                 5623.611187
        Doctor                  5332.462694
        High School or Below    6039.553187
        Master                  5729.855012
M       Bachelor                5548.031892
        College                 6005.847375
        Doctor                  5577.669457
        High School or Below    6286.731006
        Master                  5579.099207
Name: Customer Lifetime Value, dtype: float64

In [68]:
df_marketing_customer.groupby(["Gender", "Education"])["Customer Lifetime Value"].min()


Gender  Education           
F       Bachelor                1904.000852
        College                 1898.683686
        Doctor                  2395.570000
        High School or Below    2144.921535
        Master                  2417.777032
M       Bachelor                1898.007675
        College                 1918.119700
        Doctor                  2267.604038
        High School or Below    1940.981221
        Master                  2272.307310
Name: Customer Lifetime Value, dtype: float64

In [70]:
df_marketing_customer.groupby(["Gender", "Education"])["Customer Lifetime Value"].max()


Gender  Education           
F       Bachelor                73225.95652
        College                 61850.18803
        Doctor                  44856.11397
        High School or Below    55277.44589
        Master                  51016.06704
M       Bachelor                67907.27050
        College                 61134.68307
        Doctor                  32677.34284
        High School or Below    83325.38119
        Master                  50568.25912
Name: Customer Lifetime Value, dtype: float64

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here